In [121]:
#Load necessary packages
import pandas as pd
import os
import numpy as np
import copy

In [122]:
#Load data created in Library_Week1_Notebook2_DataCleanUp

#Load data
os.chdir("//Users/Olga/Documents/INSIGHT2019/Library data/AllPldData")
Files_in_folder = os.listdir()
File_names = ['_2014', '_2015', '_2016', '_2001', '_2002', '_2003', 
              '_2004','_2005', '_2006', '_2007', '_2008', '_2009', 
              '_2010', '_2011', '_2012', '_2013', '_2000', '_1992', 
              '_1993', '_1994', '_1995', '_1996', '_1997', '_1998', '_1999']
Files = []

for filename in os.listdir():
    if filename.endswith('csv'):
        Files.append(pd.read_csv(filename, encoding = 'latin-1', low_memory = False))
        
#create dictionary of libraries labeled by year
LibData_dict = {}
for i in range(0, len(File_names)):
    LibData_dict[File_names[i]] = Files[i]
    
for k, v in LibData_dict.items():
    print(k, v.shape, type(LibData_dict[k]))

ColumnsPresent = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict.items():
    dataframe_to_join = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent = ColumnsPresent.join(dataframe_to_join, how = 'outer')

#Rearrange columns in dataframe. 
cols = ColumnsPresent.columns.tolist()
cols_rearr = cols[17:] + cols[0:17]
cols_rearr2 = cols_rearr[0:8] + cols_rearr[24:25] + cols_rearr[11:24] + cols_rearr[8:11]

ColumnsPresent_rearr = ColumnsPresent[cols_rearr2]

ColumnsPresent_rearr_str = ColumnsPresent_rearr.applymap(str)
ColumnsPresent_rearr_str = ColumnsPresent_rearr_str.reset_index(drop = True)

#create DF of unique column values
UniqueLabels = np.unique(ColumnsPresent_rearr_str.values)
LabelsBoolDF = pd.DataFrame(UniqueLabels)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_rearr_str.columns)):
    lis = []
    for i in range(len(UniqueLabels)):
        lis.append(ColumnsPresent_rearr_str.iloc[:, j].isin([UniqueLabels[i]]).any())
    LabelsBoolDF[ColumnsPresent_rearr_str.columns[j]] = lis
    
## Sum all TRUE values. Value of 25 = all true
sum_list = []
for i in range(LabelsBoolDF.shape[0]):
    sum_list.append(sum(LabelsBoolDF.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF['TRUE SUM'] = sum_list
LabelsBoolDF['Notes'] = ""
LabelsBoolDF.loc[LabelsBoolDF[LabelsBoolDF['TRUE SUM']==25].index, 'Notes'] = "OK"

#rows in which not all TRUE (meaning this column/data does not exist for all year)
MissingData = LabelsBoolDF[LabelsBoolDF['TRUE SUM']!=25].index.tolist()

#Columns in which data is unavailable for all years.
LabelsBoolDF.iloc[1,27] = "OK - Mailing address. Not necessary for analysis."
LabelsBoolDF.iloc[2,27] = "OK - Address type. Not necessary for analysis."
LabelsBoolDF.iloc[3,27] = "ATTEND was called VISITS after 1998. Combine with VISITS."
LabelsBoolDF.iloc[4:7,27] = "AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
LabelsBoolDF.iloc[12,27] = "Capital revenue, only computed after 2003. No equivalent prior, it seems."
LabelsBoolDF.iloc[13,27] = "OK - Call Based Statistical Area. Not necessary for analysis"
LabelsBoolDF.iloc[14,27] = "OK - Congressional district code. Not necessary for analysis."
LabelsBoolDF.iloc[15,27] = "OK - Census block. Not necessary for analysis."
LabelsBoolDF.iloc[17,27] = "OK - Census track. Not necessary for analysis."
LabelsBoolDF.iloc[19,27] = "OK - CITY_M. Not necessary for analysis."
LabelsBoolDF.iloc[21,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[22,27] = "County population. Incomplete. Probably OK. Just need to make sure the there is another better measure of population - POPU_LSA I think."
LabelsBoolDF.iloc[24,27] = "OK - FSCS Public Library Definition. Not necessary for analysis."
LabelsBoolDF.iloc[25,27] = "C_LEGBAS - Legal Basis Code. SAME as C_LEGBASE 1998 and ON. Combine the two rows."
LabelsBoolDF.iloc[26,27] = "C_LEGBASE - Legal Basis Code. SAME as C_LEGBAS 1997 and prior. Combine the two rows."
LabelsBoolDF.iloc[28,27] = "DATABASE - Total licensed databases."
LabelsBoolDF.iloc[29,27] = "DB_LOC - Local licensed databases"
LabelsBoolDF.iloc[30,27] = "DB_LO_OT - Local/other databases"
LabelsBoolDF.iloc[31:33,27] = "More database info"
LabelsBoolDF.iloc[33,27] = "DUPLI. Called HRS_OPEN 1998 ON. Combine with HRS_OPEN."
LabelsBoolDF.iloc[34,27] = "EBOOK. Starts 2003 (makes sense)"
LabelsBoolDF.iloc[35,27] = "EC_LO_OT. Local/other electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[36,27] = "EC_ST. State electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[37,27] = "ELACCEXP. Operating expendature for electronic access."
LabelsBoolDF.iloc[38,27] = "ELCONT. 2016 only [2016 doc: first collection]. Electronic content use. Total circulation."
LabelsBoolDF.iloc[39,27] = "ELECCOLL. Total electronic collections. 2015, 2016 only."
LabelsBoolDF.iloc[40,27] = "ELINFO. Retrival of electronic infomation. 2016 only. "
LabelsBoolDF.iloc[41,27] = "ELMATCIRC. Circulation of electronic material. 2013-2016."
LabelsBoolDF.iloc[42,27] = "ELMATEXP. 1995-2016. Circulation of electronic materials. This is a good one to use!"
LabelsBoolDF.iloc[43,27] = "ELMATS. Number of library materials in electronic format. 1995-2002"
LabelsBoolDF.iloc[44,27] = "ELSVCACC. Library access to electronic services. Yes, no. Unnecessary. 1995-2002."
LabelsBoolDF.iloc[45,27] = "OK - ENDDATE. Not necessary for analysis."
LabelsBoolDF.iloc[46,27] = "ERES_USR. 1999-2005. Electronic resource users per year. "
LabelsBoolDF.iloc[47,27] = "ESUBSCRP. 2003-2009. Electronic subscriptions."
LabelsBoolDF.iloc[48,27] = "FCAP_REV. Federal gov capital revenue. 2005 and on."
LabelsBoolDF.iloc[50:61,27] = "OK - FILL. Not necessary for analysis."
LabelsBoolDF.iloc[61,27] = "OK - county code. Not necessary for analysis."
LabelsBoolDF.iloc[62,27] = "OK - place code. Not necessary for analysis."
LabelsBoolDF.iloc[63,27] = "OK - state code. Not necessary for analysis."
LabelsBoolDF.iloc[65:133,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[133,27] = "OK - GAL. Not necessary for analysis. Geocode location."
LabelsBoolDF.iloc[134,27] = "OK - GALMS. Not necessary for analysis."
LabelsBoolDF.iloc[135,27] = "GEOCODE - Geographical code. Might be useful. Can add to old datasets that don't have it."
LabelsBoolDF.iloc[136,27] = "OK - ID. Not necessary."
LabelsBoolDF.iloc[137,27] = "GPTERMS. Internet computers used by general public. 1998-2016."
LabelsBoolDF.iloc[138,27] = "HRS_OPEN. Called DUPLI 1997 and prior. Combine with DUPLI."
LabelsBoolDF.iloc[139:181,27] = "OK - imputation flags. Not necessary for analysis."
LabelsBoolDF.iloc[181,27] = "OK - County code. Not necassary for analysis."
LabelsBoolDF.iloc[182,27] = "OK - code. Not necassary for analysis."
LabelsBoolDF.iloc[183,27] = "INETACC - Internet access."
LabelsBoolDF.iloc[184,27] = "OK - imputation flag"
LabelsBoolDF.iloc[185,27] = "Kid program attendance. Combine with KIDATTEN."
LabelsBoolDF.iloc[186,27] = "Kid program attendance. Combine with KIDATTEND."
LabelsBoolDF.iloc[188,27] = "KIDPRO - Total kids program attendance."
LabelsBoolDF.iloc[189,27] = "OK - LATITUDE. NOt necessary for analysis."
LabelsBoolDF.iloc[190,27] = "LCAP_REV. Local gov capital revenue."
LabelsBoolDF.iloc[193,27] = "LIBRARIAN. Combine with LIBRARIA."
LabelsBoolDF.iloc[194,27] = "LIBRARIA. Combine with LIBRARIAN."
LabelsBoolDF.iloc[197,27] = "Urban-centric locale code. 2008-2016. 2008 doc: started in 2008. Can add to older years if necessary."
LabelsBoolDF.iloc[199,27] = "OK - LOGITUDE. Not necessary for analysis."
LabelsBoolDF.iloc[200,27] = "OK - LSABOUND. Legal services boundary. Not necessary for analysis."
LabelsBoolDF.iloc[202,27] = "OK - MAT_CENT. Match centroid. Not necessary for analysis."
LabelsBoolDF.iloc[203,27] = "OK - MAT_TYPE. Match status code. Not necessary for analysis."
LabelsBoolDF.iloc[204,27] = "OK - MICROF. Metro, micropolitan. Not necessary for analysis."
LabelsBoolDF.iloc[205,27] = "OK - MSTATUS. Match status. Not necessary for analysis." 
LabelsBoolDF.iloc[207,27] = "OCAP_REV. Other capital revenue."
LabelsBoolDF.iloc[209,27] = "OTHMATEX. Operating expenditures for all other library materials."
LabelsBoolDF.iloc[213,27] = "PHYSCIR. First collected in 2016."
LabelsBoolDF.iloc[214,27] = "PITUSR. Public use of internet."
LabelsBoolDF.iloc[215,27] = "POPU. Combine with POPU_LSA."
LabelsBoolDF.iloc[216,27] = "POPU_LSA. Population of legal service area. Combine with POPU."
LabelsBoolDF.iloc[217,27] = "POPU_UND. Combine with POPU_UNDUP."
LabelsBoolDF.iloc[218,27] = "POPU_UNDUP. Combine with POPU_UND."
LabelsBoolDF.iloc[219,27] = "OK - POSTMS. Postal match status. Unnecessary for analysis."
LabelsBoolDF.iloc[220,27] = "PRIMATEXP - Print material expenditures."
LabelsBoolDF.iloc[221,27] = "PUB_FIPS - OK. State code. Not necessary for analysis."
LabelsBoolDF.iloc[222,27] = "REAPLOCALE - urbancentric location code. Only 2016. Anything similar used before?"
LabelsBoolDF.iloc[223,27] = "REFERENC. Combine with REFERENCE."
LabelsBoolDF.iloc[224,27] = "REFERENCE. Combine with REFERENC."
LabelsBoolDF.iloc[225,27] = "REGBOR. Registered users. Anything similar prior to 2006?"
LabelsBoolDF.iloc[228,27] = "SCAP_REV. State revenue."
LabelsBoolDF.iloc[229,27] = "OK - SCORE. Address match. Not necessary for analysis."
LabelsBoolDF.iloc[231,27] = "STAFFEXP - Total staff exp. Salary and benefits. Combine with TOTEXP."
LabelsBoolDF.iloc[232,27] = "OK - STARTDAT. Reporting period start date. Not necessary for analysis."
LabelsBoolDF.iloc[233,27] = "OK - STATADDR. Address change code. Not necessary for analysis."
LabelsBoolDF.iloc[234,27] = "OK - STATNAME. Name change code. Not necessary for analysis."
LabelsBoolDF.iloc[235,27] = "OK - STATSTRU. Structure change code. Not necessary for analysis."
LabelsBoolDF.iloc[236,27] = "OK - STFTERMS. Internal terminals used by staff only. Not necessary for analysis."
LabelsBoolDF.iloc[238,27] = "SUBSCRIP. Combine with SUBSCRIPT."
LabelsBoolDF.iloc[239,27] = "SUBSCRIPT. Combine with SUBSCRIP."
LabelsBoolDF.iloc[240,27] = "TOTATTEN. Total attendance at library programs."
LabelsBoolDF.iloc[242,27] = "TOTCOLL. Total collection use. Only 2016."
LabelsBoolDF.iloc[243,27] = "TOTEXP. TOtal staff expenditures. Salary and benefits. Combine with STAFFEXP."
LabelsBoolDF.iloc[244,27] = "TOTEXPCO. Total expenditure of lib collection. Combine with TOTEXPCOL."
LabelsBoolDF.iloc[245,27] = "TOTEXPCOL. Total expenditure of lib collection. Combine with TOTEXPCO."
LabelsBoolDF.iloc[247,27] = "TOTOPEXP. Total operating expenditures. Combine with TOTOPEXP1."
LabelsBoolDF.iloc[248,27] = "TOTOPEXP1. Total operating expenditures. Combine with TOTOPEXP."
LabelsBoolDF.iloc[249,27] = "TOTPEMP - Total paid FTE employees. Combine with TOTSTAFF."
LabelsBoolDF.iloc[250,27] = "TOTPRO - Total library programs. Can this be combined with another label?"
LabelsBoolDF.iloc[251,27] = "TOTSTAFF. Total paid FTE employees. Combine with TOTPEMP."
LabelsBoolDF.iloc[252:255,27] = "VIDEO was later split into VIDEO_DL and VIDEO_PH."
LabelsBoolDF.iloc[255,27] = "VISITS. Combine with ATTEND."
LabelsBoolDF.iloc[256,27] = "OK - WEB_ADDR. Not necessary for data."
LabelsBoolDF.iloc[257,27] = "WIFISESS. Any similar metric prior?"
LabelsBoolDF.iloc[258,27] = "YAATTEN. Young adult program attendance."
LabelsBoolDF.iloc[259,27] = "YAPRO. Total young adult programs."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[260:262,27] = "OK - Submission year.  YR and YR_SUB are the same. Not necessary for analysis."
LabelsBoolDF.iloc[262,27] = "ZIP. Combine with ZIP1."
LabelsBoolDF.iloc[263,27] = "ZIP1. Combine with ZIP."
LabelsBoolDF.iloc[264:268,27] = "OK - More zip information. Not necessary for analysis."
LabelsBoolDF.iloc[268,27] = "Note that lack of nan."

#Column names to keep
#LabelsBoolDF.loc[[0, 3:13, 15], 0].tolist()
Cols_to_keep_idx = [0]
Cols_to_keep_idx.extend(range(3,13))
Cols_to_keep_idx.extend((16, 18, 20, 25, 26))
Cols_to_keep_idx.extend(range(33, 49))
Cols_to_keep_idx.extend((64, 135, 137, 138, 183))
Cols_to_keep_idx.extend(range(185, 189))
Cols_to_keep_idx.extend(range(191, 199))
Cols_to_keep_idx.append(201)
Cols_to_keep_idx.extend(range(206, 219))
Cols_to_keep_idx.append(220)
Cols_to_keep_idx.extend(range(222, 229))
Cols_to_keep_idx.extend((230, 231, 237))
Cols_to_keep_idx.extend(range(237, 256))
Cols_to_keep_idx.extend(range(257, 260))
Cols_to_keep_idx.extend((262, 263))

Cols_to_keep_in_lib_dfs = LabelsBoolDF.iloc[Cols_to_keep_idx, 0].tolist()

#Make dictionaries of smaller dataframe with Cols_to_keep_in_lib_dfs ONLY
LibData_dict_reduced = dict()

for k, v in LibData_dict.items():
    LibData_dict_reduced[k] = pd.DataFrame()
    for i in Cols_to_keep_in_lib_dfs:
        if i in v:
            TempList = list(v[i])
            LibData_dict_reduced[k][i] = TempList

#Did above loop do what I thought it did?
print("Original dfs:")
for k, v in LibData_dict.items():
    print(k, v.shape)

print("\n", "Reduced dfs:")  
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Combine col names that mean same thing in different years into single 
Names_to_combine = ["ATTEND", "VISITS",  "C_LEGBAS", "C_LEGBASE", "DUPLI", "HRS_OPEN", "KIDATTEN", "KIDATTEND", 
                    "LIBRARIAN", "LIBRARIA", "POPU", "POPU_LSA", "POPU_UND", "POPU_UNDUP", "REFERENC", "REFERENCE", 
                    "STAFFEXP", "TOTEXP", "SUBSCRIP", "SUBSCRIPT", "TOTEXPCO", "TOTEXPCOL", 
                    "TOTOPEXP", "TOTOPEXP1", "TOTPEMP", "TOTSTAFF", "ZIP", "ZIP1"]

for i in range(0, len(Names_to_combine)-1, 2):
    for k, v in LibData_dict_reduced.items(): 
        if Names_to_combine[i] in v:
            name = Names_to_combine[i]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name: newname}, inplace = True)
        elif Names_to_combine[i+1] in v:
            name = Names_to_combine[i + 1]
            newname = Names_to_combine[i] + "-" + Names_to_combine[i+1]
            v = v.rename(columns = {name : newname}, inplace = True)

#Check that above code worked - names changes
for k, v in LibData_dict_reduced.items():
    print(v.columns)

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

#Columns to add to DFs
"AUDIO is divided into AUDIO_PH, AUDIO_DL starting 2010. Source: 2010 Documentation."
"VIDEO was later split into VIDEO_DL and VIDEO_PH."

for k, v in LibData_dict_reduced.items(): 
    if "AUDIO_DL" in v:
        v["AUDIO"] = v['AUDIO_DL'] + v['AUDIO_PH']
    if "VIDEO_DL" in v:
        v["VIDEO"] = v['VIDEO_DL'] + v['VIDEO_PH']

#Check that # columns in some dataframes increased by 2 (expected)
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Create DF w boolean values for prensence of each unique value in REDUCED dataframes
#THIS IS THE END GOAL OF THIS DATA

#Get all unique columns present in LibData_dict_reduced dataframes. 
ColumnsPresent_reduced = pd.DataFrame(index=range(0),columns=range(0))

for k, v in LibData_dict_reduced.items():
    dataframe_to_join_reduced = pd.DataFrame({k: list(v.columns)})
    ColumnsPresent_reduced = ColumnsPresent_reduced.join(dataframe_to_join_reduced, how = 'outer')

ColumnsPresent_reduced_str = ColumnsPresent_reduced.applymap(str)
ColumnsPresent_reduced_str = ColumnsPresent_reduced_str.reset_index(drop = True)
    
UniqueLabels_reduced = np.unique(ColumnsPresent_reduced_str.values)
LabelsBoolDF_reduced = pd.DataFrame(UniqueLabels_reduced)

#Create DF w boolean values for prensence of each unique value
#THIS IS THE END GOAL OF THIS DATA
for j in range(len(ColumnsPresent_reduced_str.columns)):
    lis = []
    for i in range(len(UniqueLabels_reduced)):
        lis.append(ColumnsPresent_reduced_str.iloc[:, j].isin([UniqueLabels_reduced[i]]).any())
    LabelsBoolDF_reduced[ColumnsPresent_reduced_str.columns[j]] = lis

## Sum all TRUE values. Value of 25 = all true
sum_list_reduced = []
for i in range(LabelsBoolDF_reduced.shape[0]):
    sum_list_reduced.append(sum(LabelsBoolDF_reduced.iloc[i, 1:26]))

#Label OK in Notes
LabelsBoolDF_reduced['TRUE SUM'] = sum_list_reduced
LabelsBoolDF_reduced['Notes'] = ""
LabelsBoolDF_reduced.loc[LabelsBoolDF_reduced[LabelsBoolDF_reduced['TRUE SUM']==25].index, 'Notes'] = "OK"

All_Cols_in_Reduced_dfs = list(LabelsBoolDF_reduced[0])
All_Cols_in_Reduced_dfs

for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)
    
#Delete columns deemed NOT useful for downstream analysis
NonUsefulColstoDelete = ['ENDDATE', 'LIBID', 'PHONE', 'POPU_UND-POPU_UNDUP', 'RSTATUS', 'TOTCOLL', 'ZIP-ZIP1']

LibData_dict_reduced_dropuseless = copy.deepcopy(LibData_dict_reduced)

for k, v in LibData_dict_reduced_dropuseless.items(): 
    for i in range(len(NonUsefulColstoDelete)):
        if NonUsefulColstoDelete[i] in v:
            v.drop([NonUsefulColstoDelete[i]], axis = 1, inplace = True)
            
#Check quickly whether columns were dropped
print("Reduced dfs:")
for k, v in LibData_dict_reduced.items():    
    print(k, v.shape)

print("\n", "Reduced and dropped useless dfs:")
for k, v in LibData_dict_reduced_dropuseless.items():    
    print(k, v.shape)

#Get rid of negative values (replace w NaN)
#For all DFs, replace (-1, -3, -9) values with NaN. 
LibData_dict_reduced_dropuseless_NegRepWNaN = copy.deepcopy(LibData_dict_reduced_dropuseless)

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    v[(v == -1) | (v == -3) | (v == -9)] = np.nan
    
#See if this worked (looks like it did)
print(sum(LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0))
print(np.where((LibData_dict_reduced_dropuseless['_2014']['AUDIO_DL'] < 0) == True))

print(sum(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))
print(np.where(LibData_dict_reduced_dropuseless_NegRepWNaN['_2014']['AUDIO_DL'].isnull()))


_2014 (9305, 159) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 159) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 151) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 108) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 107) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 110) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 128) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 131) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 139) <class 'pandas.core.frame.DataFrame'>
_2007 (9217, 148) <class 'pandas.core.frame.DataFrame'>
_2008 (9284, 150) <class 'pandas.core.frame.DataFrame'>
_2009 (9299, 152) <class 'pandas.core.frame.DataFrame'>
_2010 (9308, 154) <class 'pandas.core.frame.DataFrame'>
_2011 (9315, 157) <class 'pandas.core.frame.DataFrame'>
_2012 (9305, 155) <class 'pandas.core.frame.DataFrame'>
_2013 (9309, 157) <class 'pandas.core.frame.DataFrame'>
_2000 (9078, 108) <class 'pandas.core.frame.DataFrame'>
_1992 (8944, 83) <class 'pandas.core.frame.DataF

_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)
Reduced dfs:
_2014 (9305, 69)
_2015 (9251, 72)
_2016 (9252, 77)
_2001 (9133, 52)
_2002 (9141, 52)
_2003 (9214, 53)
_2004 (9210, 56)
_2005 (9201, 60)
_2006 (9211, 61)
_2007 (9217, 61)
_2008 (9284, 62)
_2009 (9299, 64)
_2010 (9308, 67)
_2011 (9315, 67)
_2012 (9305, 67)
_2013 (9309, 68)
_2000 (9078, 52)
_1992 (8944, 44)
_1993 (8929, 44)
_1994 (8920, 44)
_1995 (8981, 49)
_1996 (8946, 49)
_1997 (8968, 49)
_1998 (8966, 51)
_1999 (9048, 52)

 Reduced and dropped useless dfs:
_2014 (9305, 63)
_2015 (9251, 66)
_2016 (9252, 70)
_2001 (9133, 47)
_2002 (9141, 47)
_2003 (9214, 48)


In [123]:
#Reduce each yearly dataframe to FSCSKEY and usage data only
LibraryData_UsageOnly = dict()

for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    LibraryData_UsageOnly[k] = v[['FSCSKEY', 'POPU-POPU_LSA', 'ATTEND-VISITS']]

In [124]:
#Check that reduction worked
for k, v in LibData_dict_reduced_dropuseless_NegRepWNaN.items():
    print(k, v.shape, type(LibData_dict_reduced_dropuseless_NegRepWNaN[k]))
    print(k, LibraryData_UsageOnly[k].shape, type(LibraryData_UsageOnly[k]))

_2014 (9305, 63) <class 'pandas.core.frame.DataFrame'>
_2014 (9305, 3) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 66) <class 'pandas.core.frame.DataFrame'>
_2015 (9251, 3) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 70) <class 'pandas.core.frame.DataFrame'>
_2016 (9252, 3) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 47) <class 'pandas.core.frame.DataFrame'>
_2001 (9133, 3) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 47) <class 'pandas.core.frame.DataFrame'>
_2002 (9141, 3) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 48) <class 'pandas.core.frame.DataFrame'>
_2003 (9214, 3) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 50) <class 'pandas.core.frame.DataFrame'>
_2004 (9210, 3) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 54) <class 'pandas.core.frame.DataFrame'>
_2005 (9201, 3) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 55) <class 'pandas.core.frame.DataFrame'>
_2006 (9211, 3) <class 'pandas.core.frame.DataFrame'>
_2007 (9217, 55) <c

In [125]:
#Add usage column to each library
for k, v in LibraryData_UsageOnly.items():
    v['Usage'] = v['ATTEND-VISITS']/v['POPU-POPU_LSA']
    
#Check that column was added
for k, v in LibraryData_UsageOnly.items():
    print(k, v.shape)

//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


_2014 (9305, 4)
_2015 (9251, 4)
_2016 (9252, 4)
_2001 (9133, 4)
_2002 (9141, 4)
_2003 (9214, 4)
_2004 (9210, 4)
_2005 (9201, 4)
_2006 (9211, 4)
_2007 (9217, 4)
_2008 (9284, 4)
_2009 (9299, 4)
_2010 (9308, 4)
_2011 (9315, 4)
_2012 (9305, 4)
_2013 (9309, 4)
_2000 (9078, 4)
_1992 (8944, 4)
_1993 (8929, 4)
_1994 (8920, 4)
_1995 (8981, 4)
_1996 (8946, 4)
_1997 (8968, 4)
_1998 (8966, 4)
_1999 (9048, 4)


In [126]:
for k, v, in LibraryData_UsageOnly.items():
    newname = "Usage" + k
    v = v.rename(columns = {"Usage": newname}, inplace = True)

//anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [127]:
#MERGE ON FSCSKEY. 
LibraryData_UsageOnly_YearsMerged = pd.DataFrame()

for k, v in LibraryData_UsageOnly.items():
    if k == '_2014':
        LibraryData_UsageOnly_YearsMerged = v
    else: 
        LibraryData_UsageOnly_YearsMerged = pd.merge(LibraryData_UsageOnly_YearsMerged, v, how = 'outer', on = 'FSCSKEY')


In [128]:
#Keep usage only
LibraryData_UsageOnly_YearsMerged = LibraryData_UsageOnly_YearsMerged.drop(columns = ['POPU-POPU_LSA_x', 'ATTEND-VISITS_x', 
                                                                                      'POPU-POPU_LSA_y', 'ATTEND-VISITS_y',
                                                                                     'POPU-POPU_LSA', 'ATTEND-VISITS'])

LibraryData_UsageOnly_YearsMerged.shape


(10158, 26)

In [129]:
#Reorganize columns by date
col_names = list(LibraryData_UsageOnly_YearsMerged.columns)
col_order = col_names[0:1] + col_names[18:26] + col_names[17:18] + col_names[4:17] + col_names[1:4]
LibraryData_UsageOnly_YearsMerged = LibraryData_UsageOnly_YearsMerged[col_order]

In [130]:
#sum(LibraryData_UsageOnly_YearsMerged['Usage_2007':'Usage_2011'])
LibraryData_UsageOnly_YearsMerged['Usage_Sum_2007_to_2011'] = LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2007':'Usage_2011'].sum(axis = 1)
LibraryData_UsageOnly_YearsMerged['Number of NaN in 2007-2011 data'] = LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2007':'Usage_2011'].isnull().sum(axis = 1)
LibraryData_UsageOnly_YearsMerged['Usage_Sum_2012_to_2016'] = LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2012':'Usage_2016'].sum(axis = 1)
LibraryData_UsageOnly_YearsMerged['Number of NaN in 2012-2016 data'] = LibraryData_UsageOnly_YearsMerged.loc[:, 'Usage_2012':'Usage_2016'].isnull().sum(axis = 1)
LibraryData_UsageOnly_YearsMerged['Usage_Sum_2007_to_2011_NORM'] = LibraryData_UsageOnly_YearsMerged['Usage_Sum_2007_to_2011']/(5-LibraryData_UsageOnly_YearsMerged['Number of NaN in 2007-2011 data'])
LibraryData_UsageOnly_YearsMerged['Usage_Sum_2012_to_2016_NORM'] = LibraryData_UsageOnly_YearsMerged['Usage_Sum_2012_to_2016']/(5-LibraryData_UsageOnly_YearsMerged['Number of NaN in 2012-2016 data'])

In [131]:
LibraryData_UsageOnly_YearsMerged

,FSCSKEY,Usage_1992,Usage_1993,Usage_1994,Usage_1995,Usage_1996,Usage_1997,Usage_1998,Usage_1999,Usage_2000,...,Usage_2013,Usage_2014,Usage_2015,Usage_2016,Usage_Sum_2007_to_2011,Number of NaN in 2007-2011 data,Usage_Sum_2012_to_2016,Number of NaN in 2012-2016 data,Usage_Sum_2007_to_2011_NORM,Usage_Sum_2012_to_2016_NORM
0,AK0001,1.840943,1.835536,1.673360,1.518761,1.595745,1.705566,1.596071,1.211584,0.836397,...,4.591377,2.612433,2.873905,3.269212,20.541945,0,17.613992,0,4.108389,3.522798
1,AK0002,3.675581,2.097850,1.649358,3.585049,3.634552,3.626281,3.385498,3.417765,3.041524,...,2.670980,3.137169,3.055469,2.878410,14.271267,0,14.217864,0,2.854253,2.843573
2,AK0003,3.491525,3.950769,9.403077,4.504792,6.740675,6.375221,6.797710,6.045802,8.514986,...,4.468085,4.181818,3.152074,4.810924,38.637771,0,21.329568,0,7.727554,4.265914
3,AK0006,6.573541,5.200080,5.989219,10.587103,9.023306,10.670748,9.728334,2.094087,3.304697,...,4.973877,7.207178,7.395649,7.170884,15.444350,0,30.660560,0,3.088870,6.132112
4,AK0007,3.796752,3.194175,2.661563,2.533226,2.552051,3.020296,3.203645,3.676684,3.657753,...,3.254871,3.383491,4.705518,6.071060,19.903602,0,21.540649,0,3.980720,4.308130
5,AK0008,12.244898,12.748299,13.194444,12.068966,5.814815,5.729167,10.120482,10.240964,11.261261,...,14.795918,16.483516,16.756757,16.186869,53.068389,0,78.092626,0,10.613678,15.618525
6,AK0011,13.360996,14.107884,45.454545,45.783133,42.666667,8.108108,7.333333,21.666667,51.460000,...,14.250000,21.437500,3.979592,7.326087,72.915884,0,68.811361,0,14.583177,13.762272
7,AK0014,18.467153,19.576642,20.641667,14.463636,22.678082,24.908333,26.970874,25.728395,23.295455,...,18.729412,17.977528,12.307692,16.881356,81.685575,0,82.487825,0,16.337115,16.497565
8,AK0015,4.128472,4.336918,4.595149,3.844732,4.467480,4.462527,4.828753,5.013699,5.395994,...,12.985663,11.352542,13.402214,11.060000,50.239691,0,59.749225,0,10.047938,11.949845
9,AK0016,9.174121,7.835383,9.442049,6.929907,7.113126,7.314957,10.112797,7.261766,7.519152,...,9.858384,10.025372,8.641965,7.916415,56.618078,0,48.123915,0,11.323616,9.624783


In [132]:
#Delete all libraries which have < 3 Usage values in 5 year period
Rows_with_toolittleusagedata = LibraryData_UsageOnly_YearsMerged[(LibraryData_UsageOnly_YearsMerged['Number of NaN in 2007-2011 data'] >= 3) | (LibraryData_UsageOnly_YearsMerged['Number of NaN in 2012-2016 data'] >= 3)].index

LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts = LibraryData_UsageOnly_YearsMerged.drop(Rows_with_toolittleusagedata)



In [133]:
LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts

,FSCSKEY,Usage_1992,Usage_1993,Usage_1994,Usage_1995,Usage_1996,Usage_1997,Usage_1998,Usage_1999,Usage_2000,...,Usage_2013,Usage_2014,Usage_2015,Usage_2016,Usage_Sum_2007_to_2011,Number of NaN in 2007-2011 data,Usage_Sum_2012_to_2016,Number of NaN in 2012-2016 data,Usage_Sum_2007_to_2011_NORM,Usage_Sum_2012_to_2016_NORM
0,AK0001,1.840943,1.835536,1.673360,1.518761,1.595745,1.705566,1.596071,1.211584,0.836397,...,4.591377,2.612433,2.873905,3.269212,20.541945,0,17.613992,0,4.108389,3.522798
1,AK0002,3.675581,2.097850,1.649358,3.585049,3.634552,3.626281,3.385498,3.417765,3.041524,...,2.670980,3.137169,3.055469,2.878410,14.271267,0,14.217864,0,2.854253,2.843573
2,AK0003,3.491525,3.950769,9.403077,4.504792,6.740675,6.375221,6.797710,6.045802,8.514986,...,4.468085,4.181818,3.152074,4.810924,38.637771,0,21.329568,0,7.727554,4.265914
3,AK0006,6.573541,5.200080,5.989219,10.587103,9.023306,10.670748,9.728334,2.094087,3.304697,...,4.973877,7.207178,7.395649,7.170884,15.444350,0,30.660560,0,3.088870,6.132112
4,AK0007,3.796752,3.194175,2.661563,2.533226,2.552051,3.020296,3.203645,3.676684,3.657753,...,3.254871,3.383491,4.705518,6.071060,19.903602,0,21.540649,0,3.980720,4.308130
5,AK0008,12.244898,12.748299,13.194444,12.068966,5.814815,5.729167,10.120482,10.240964,11.261261,...,14.795918,16.483516,16.756757,16.186869,53.068389,0,78.092626,0,10.613678,15.618525
6,AK0011,13.360996,14.107884,45.454545,45.783133,42.666667,8.108108,7.333333,21.666667,51.460000,...,14.250000,21.437500,3.979592,7.326087,72.915884,0,68.811361,0,14.583177,13.762272
7,AK0014,18.467153,19.576642,20.641667,14.463636,22.678082,24.908333,26.970874,25.728395,23.295455,...,18.729412,17.977528,12.307692,16.881356,81.685575,0,82.487825,0,16.337115,16.497565
8,AK0015,4.128472,4.336918,4.595149,3.844732,4.467480,4.462527,4.828753,5.013699,5.395994,...,12.985663,11.352542,13.402214,11.060000,50.239691,0,59.749225,0,10.047938,11.949845
9,AK0016,9.174121,7.835383,9.442049,6.929907,7.113126,7.314957,10.112797,7.261766,7.519152,...,9.858384,10.025372,8.641965,7.916415,56.618078,0,48.123915,0,11.323616,9.624783


In [134]:
#Quantify change in usage (Last 5 year - Previous 5 year/Previous 5 year)
LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts['UsageChange'] =  ((LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts['Usage_Sum_2012_to_2016_NORM'])-(LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts['Usage_Sum_2007_to_2011_NORM']))/(LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts['Usage_Sum_2007_to_2011_NORM'])

#Categorize change
Negative_change_index = LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts[LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts.UsageChange < 0].index
Positive_change_index = LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts[LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts.UsageChange > 0].index

LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts['UsageChange_0or1'] = ""
LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts.loc[Negative_change_index, ['UsageChange_0or1']] = 0 
LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts.loc[Positive_change_index, ['UsageChange_0or1']] = 1    
                                                                       

In [135]:
#Check that every value got 0 or 1. 
print(len(Negative_change_index) + len(Positive_change_index) == LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts.shape[0])

True


In [148]:
#Delete unnecessary columns from 2016 DF. 

#Delete city - too many unique value; will not provide much data
print(LibData_dict_reduced_dropuseless_NegRepWNaN['_2016'].loc[:, ['CITY']].nunique())
#COUNTY may give useful info - consider categorizing. 
print(LibData_dict_reduced_dropuseless_NegRepWNaN['_2016'].loc[:, ['CNTY']].nunique())

#Delete LIBNAME, Delete ADDRESS (Unique, not useful), Delete CITY (Almost unique, not useful)
_2016DataCleanedforLogReg = LibData_dict_reduced_dropuseless_NegRepWNaN['_2016'].drop(['ADDRESS', 'CITY', 'LIBNAME'], axis=1)



#'C_LEGBAS-C_LEGBASE', 'STABR', 'GEOCODE', 'CNTY'


CITY    6319
dtype: int64
CNTY    1659
dtype: int64


In [154]:
#Make dummies for non-continuous variables
C_LEGBASE_dummies = pd.get_dummies(_2016DataCleanedforLogReg['C_LEGBAS-C_LEGBASE'], prefix = 'C_LEGBAS-C_LEGBASE')
STABR_dummies = pd.get_dummies(_2016DataCleanedforLogReg['STABR'], prefix = 'STABR')
GEOCODE_dummies = pd.get_dummies(_2016DataCleanedforLogReg['GEOCODE'], prefix = 'GEOCODE')
CNTY_dummies = pd.get_dummies(_2016DataCleanedforLogReg['CNTY'], prefix = 'CNTY')

#Delete variables to categorize
_2016DataCleanedforLogReg_Categorize = _2016DataCleanedforLogReg.drop(['C_LEGBAS-C_LEGBASE', 'STABR', 'GEOCODE', 'CNTY'], axis=1)

#Concatenate dummy variable dataframes to cleaned dataframe.
_2016DataCleanedforLogReg_Categorize_FINAL = pd.concat([_2016DataCleanedforLogReg_Categorize, C_LEGBASE_dummies, GEOCODE_dummies, CNTY_dummies], axis=1)


In [157]:
#Combine usage (encoded as 0, 1) with 2016 dataframe 
#Append to resulting 2016 DF
_2016DataCleanedforLogReg_Categorize_FINAL

#Merge ON FSCSKEY
(LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts['UsageChange_0or1'])


KeyError: 'GEOCODE'

In [158]:
LibraryData_UsageOnly_YearsMerged_DropTooFewDataPts

,FSCSKEY,Usage_1992,Usage_1993,Usage_1994,Usage_1995,Usage_1996,Usage_1997,Usage_1998,Usage_1999,Usage_2000,...,Usage_2015,Usage_2016,Usage_Sum_2007_to_2011,Number of NaN in 2007-2011 data,Usage_Sum_2012_to_2016,Number of NaN in 2012-2016 data,Usage_Sum_2007_to_2011_NORM,Usage_Sum_2012_to_2016_NORM,UsageChange,UsageChange_0or1
0,AK0001,1.840943,1.835536,1.673360,1.518761,1.595745,1.705566,1.596071,1.211584,0.836397,...,2.873905,3.269212,20.541945,0,17.613992,0,4.108389,3.522798,-0.142535,0
1,AK0002,3.675581,2.097850,1.649358,3.585049,3.634552,3.626281,3.385498,3.417765,3.041524,...,3.055469,2.878410,14.271267,0,14.217864,0,2.854253,2.843573,-0.003742,0
2,AK0003,3.491525,3.950769,9.403077,4.504792,6.740675,6.375221,6.797710,6.045802,8.514986,...,3.152074,4.810924,38.637771,0,21.329568,0,7.727554,4.265914,-0.447961,0
3,AK0006,6.573541,5.200080,5.989219,10.587103,9.023306,10.670748,9.728334,2.094087,3.304697,...,7.395649,7.170884,15.444350,0,30.660560,0,3.088870,6.132112,0.985228,1
4,AK0007,3.796752,3.194175,2.661563,2.533226,2.552051,3.020296,3.203645,3.676684,3.657753,...,4.705518,6.071060,19.903602,0,21.540649,0,3.980720,4.308130,0.082249,1
5,AK0008,12.244898,12.748299,13.194444,12.068966,5.814815,5.729167,10.120482,10.240964,11.261261,...,16.756757,16.186869,53.068389,0,78.092626,0,10.613678,15.618525,0.471547,1
6,AK0011,13.360996,14.107884,45.454545,45.783133,42.666667,8.108108,7.333333,21.666667,51.460000,...,3.979592,7.326087,72.915884,0,68.811361,0,14.583177,13.762272,-0.056291,0
7,AK0014,18.467153,19.576642,20.641667,14.463636,22.678082,24.908333,26.970874,25.728395,23.295455,...,12.307692,16.881356,81.685575,0,82.487825,0,16.337115,16.497565,0.009821,1
8,AK0015,4.128472,4.336918,4.595149,3.844732,4.467480,4.462527,4.828753,5.013699,5.395994,...,13.402214,11.060000,50.239691,0,59.749225,0,10.047938,11.949845,0.189283,1
9,AK0016,9.174121,7.835383,9.442049,6.929907,7.113126,7.314957,10.112797,7.261766,7.519152,...,8.641965,7.916415,56.618078,0,48.123915,0,11.323616,9.624783,-0.150026,0
